In [1]:
import spacy
import pandas as pd

In [2]:
## SETTINGS ##
size = 'sample'
#size = 'full'

#type = 'train'
type = 'test'

In [ ]:
ner = spacy.load("en_core_web_sm")
df = pd.read_csv('output_test.csv')

if size == 'sample':
    df = df.head(1000)
    df = df[['label', 'statement']]
    df['label'] = df['label'].isin([3, 4, 5]).astype(int)
    df.head()
else:
    df = df[['label', 'statement']]
    df['label'] = df['label'].isin([3, 4, 5]).astype(int)
    df.head()

,statement,label,label_binary,A_raw_entities,B_raw_entities
0,"90 percent of Americans ""support universal bac...",5,1,"[{'entity': 'MISC', 'score': 0.99866974, 'inde...","[{'word': '90 percent', 'entity': 'PERCENT'}, ..."
1,Last year was one of the deadliest years ever ...,1,0,[],"[{'word': 'Last year', 'entity': 'DATE'}, {'wo..."
2,"Bernie Sanders's plan is ""to raise your taxes ...",0,0,"[{'entity': 'PER', 'score': 0.9983652, 'index'...","[{'word': ""Bernie Sanders's"", 'entity': 'PERSO..."
3,Voter ID is supported by an overwhelming major...,4,1,"[{'entity': 'MISC', 'score': 0.9153446, 'index...","[{'word': 'NYers', 'entity': 'ORG'}]"
4,"Says Barack Obama ""robbed Medicare (of) $716 b...",2,0,"[{'entity': 'PER', 'score': 0.9980445, 'index'...","[{'word': 'Barack Obama', 'entity': 'PERSON'},..."


In [ ]:
def extract_entities(text):
    doc = ner(str(text))
    return [(ent.text, ent.label_) for ent in doc.ents]

# Apply NER
df["named_entities"] = df["statement"].apply(extract_entities)

#for i in range(len(df['named_entities'])):
#    print(df['named_entities'][i])

for i in range(10):
    print(df['named_entities'][i])


def tag_statement(text):
    doc = ner(str(text)) 
    tagged = text 
    offset = 0 # how much extra length we added to string

    for ent in doc.ents:
        # make opening and closing tags
        start_tag = f"[{ent.label_}]"
        end_tag = f"[/{ent.label_}]"

        start = ent.start_char + offset
        end = ent.end_char + offset

        # <TAG>entity</TAG>
        tagged = tagged[:start] + start_tag + ent.text + end_tag + tagged[end:]
        offset += len(start_tag) + len(end_tag)

    return tagged

df['tagged_statement'] = df['statement'].apply(tag_statement)

df.to_csv('data/tagged_statements_bert.csv', index=False)

""" if size == 'sample':
    if type == 'train':
        df.to_csv('tagged_statements_sample.csv', index=False)
        print('Saving sample train dataset!')
    else:
        df.to_csv('tagged_statements_sample_test.csv', index=False)  
        print('Saving sample test dataset!')
else:
    if type == 'train':
        df.to_csv('tagged_statements_train.csv', index=False)
        print('Saving train dataset!')
    else:
        df.to_csv('tagged_statements_test.csv', index=False)  
        print('Saving test dataset!') """

[('Three', 'CARDINAL'), ('the same week', 'DATE'), ('fourth', 'ORDINAL')]
[('Joe Biden', 'PERSON')]
[('Joe Biden', 'PERSON'), ('Ukrainian', 'NORP'), ('Volodymyr Zelenskyy', 'PERSON')]
[('50,000', 'MONEY'), ('Obamacare', 'NORP'), ('the next 10 years', 'DATE')]
[('The Federal Register - which', 'ORG'), ('Washington', 'GPE'), ('34,000', 'CARDINAL'), ('more than 340 pounds', 'MONEY')]
[('2009', 'DATE'), ('Texas House', 'ORG'), ('Joe Straus', 'PERSON'), ('Planned Parenthood', 'ORG')]
[('America', 'GPE')]
[('Wisconsin', 'GPE'), ('Scott Walker', 'PERSON'), ('Department of Motor Vehicles', 'ORG')]
[('North Carolina', 'GPE')]
[('Republican', 'NORP'), ('Bill Flores', 'PERSON')]
Saving sample test dataset!
